In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from utils import to_grayscale_then_rgb

In [11]:
file_train_b_csv='..\Iris_train_basic_seg_list.csv'
df_train_basic= pd.read_csv(file_train_b_csv)
datagen=ImageDataGenerator(rescale=1./255.,preprocessing_function=to_grayscale_then_rgb)
df_train_basic['Label']= df_train_basic['Label'].astype(str)
df_train_basic


,Name,Label
0,110160_OD_INF_SANO.jpg,0
1,110160_OD_NAS_SANO.jpg,0
2,110160_OD_SUP_SANO.jpg,0
3,110160_OD_TEM_SANO.jpg,0
4,2581150_OD_INF_CMV.jpg,1
...,...,...
123,781208_OI_TEM_SANO.jpg,0
124,795004_OI_INF_POSTNER.jpg,1
125,795004_OI_NAS_POSTNER.jpg,1
126,795004_OI_SUP_POSTNER.jpg,1


In [13]:
file_val_csv='..\Iris_val_basic_seg_list.csv'
df_val= pd.read_csv(file_val_csv)
df_val['Label']= df_val['Label'].astype(str)


In [17]:

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_basic,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\train_basic",
    x_col="Name",
    y_col="Label",
    batch_size=5,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(540, 810))


Found 128 validated image filenames belonging to 2 classes.


In [18]:
val_generator = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\val",
    x_col="Name",
    y_col="Label",
    batch_size=5,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(540, 810))


Found 32 validated image filenames belonging to 2 classes.


In [6]:
file_test_csv='..\Iris_test_seg_list.csv'
df_test= pd.read_csv(file_test_csv)
df_test['class_label']= df_test['class_label'].astype(str)
df_test['label_OI']= df_test['label_OI'].astype(str)
df_test['label_OD']= df_test['label_OD'].astype(str)
test_generator_OI = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\test",
    x_col="OI",
    y_col="label_OI",
    batch_size=2,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(540, 810))

test_generator_OD = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\test",
    x_col="OD",
    y_col="label_OD",
    batch_size=2,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(540, 810))

Found 20 validated image filenames.
Found 20 validated image filenames.


In [19]:
from model import build_LeNet5
input_shape=(540,810,1)
model=build_LeNet5(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [20]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator_OI.n//test_generator_OI.batch_size
model.reset_states()

In [21]:
hist=model.fit(x=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=val_generator, 
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,verbose=1)

Epoch 1/50
25/25 [==============================] - 50s 2s/step - loss: 0.6923 - accuracy: 0.5203 - val_loss: 0.6879 - val_accuracy: 0.5667
Epoch 2/50
25/25 [==============================] - 51s 2s/step - loss: 0.6888 - accuracy: 0.5600 - val_loss: 0.6900 - val_accuracy: 0.5000
Epoch 3/50
25/25 [==============================] - 29s 1s/step - loss: 0.6855 - accuracy: 0.6016 - val_loss: 0.6928 - val_accuracy: 0.4333
Epoch 4/50
25/25 [==============================] - 32s 1s/step - loss: 0.6823 - accuracy: 0.6423 - val_loss: 0.6912 - val_accuracy: 0.4667
Epoch 5/50
25/25 [==============================] - 36s 1s/step - loss: 0.6806 - accuracy: 0.6260 - val_loss: 0.6944 - val_accuracy: 0.4667
Epoch 6/50
25/25 [==============================] - 45s 2s/step - loss: 0.6771 - accuracy: 0.6341 - val_loss: 0.6940 - val_accuracy: 0.4333
Epoch 7/50
25/25 [==============================] - 50s 2s/step - loss: 0.6733 - accuracy: 0.6504 - val_loss: 0.6943 - val_accuracy: 0.4333
Epoch 8/50
25/25 [==

KeyboardInterrupt: 

In [ ]:
model.evaluate(x=test_generator_OI,
steps=STEP_SIZE_TEST)


In [ ]:
model.fit(x=train_generator_OD, steps_per_epoch=None, validation_data=val_generator_OD, 
                    validation_steps=None,
                    epochs=7,verbose=1)

In [ ]:
model.evaluate(x=test_generator_OD,
steps=None)

In [ ]:
test_generator_OD.filenames
